# Preprocessing Chat History with `TransformMessages`

### This notebook illustrates how to handle long contexts, handle sensitive data, and more by giving the `TransfromMessages` ability to any `ConversableAgent`.

We first need to make sure you have the right dependencies installed. If you are installing autogen from source, run the following command inside the base directory:

`pip install -e .`

If you're not installing autogen from source, run the following command:

`pip install pyautogen`

In [1]:
import os
import pprint

import autogen
from autogen.agentchat.contrib.capabilities import transform_messages

In [2]:
# Define your llm config
llm_config = {
    "config_list": [{"model": "gpt-3.5-turbo", "api_key": os.environ.get("OAI_API_KEY")}],
}

In [3]:
# Define your agent; the user proxy and an assistant
assistant = autogen.AssistantAgent(
    "assistant",
    llm_config=llm_config,
)
user_proxy = autogen.UserProxyAgent(
    "user_proxy",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: "TERMINATE" in x.get("content", ""),
    max_consecutive_auto_reply=10,
)

### Handling Long Contexts

Imagine a scenario where the LLM generates an extensive amount of text, surpassing the token limit imposed by your API provider. To address this issue, you can leverage `TransformMessages` along with its constituent transformations, `MaxMessagesTransform` and `TruncateMessageTransform`.

- `MaxMessagesTransform`: allows you to restrict the total number of messages considered as context history. This capability is particularly useful when you want to limit the conversational context to a specific number of recent messages, ensuring efficient processing and response generation.
- `TruncateMessageTransform`: enables you to cap the total number of tokens, either on a per-message basis or across the entire context history. This transformation is invaluable when you need to adhere to strict token limits imposed by your API provider, preventing unnecessary costs or errors caused by exceeding the allowed token count.
  

In [4]:
# Limit the message history to the 3 most recent messages
max_msg_transfrom = transform_messages.MaxMessagesTransform(max_messages=3)

# Limit the token limit per message to 10 tokens
token_limit_transform = transform_messages.TokenLimitTransform(max_tokens_per_message=10)

# Create the ability and add it to the assistant
msg_transform_ability = transform_messages.TransformMessages(transforms=[max_msg_transfrom, token_limit_transform])
msg_transform_ability.add_to_agent(assistant)

In [5]:
# Let's send few messages to the assistant
for i in range(10):
    user_proxy.send(f"Message num: {i}", assistant, request_reply=False, silent=True)
user_proxy.send(f"Message num: {10}", assistant, request_reply=True, silent=True)

Len of messages: 11
[{'content': 'Message num: 0', 'role': 'user'}, {'content': 'Message num: 1', 'role': 'user'}, {'content': 'Message num: 2', 'role': 'user'}, {'content': 'Message num: 3', 'role': 'user'}, {'content': 'Message num: 4', 'role': 'user'}, {'content': 'Message num: 5', 'role': 'user'}, {'content': 'Message num: 6', 'role': 'user'}, {'content': 'Message num: 7', 'role': 'user'}, {'content': 'Message num: 8', 'role': 'user'}, {'content': 'Message num: 9', 'role': 'user'}, {'content': 'Message num: 10', 'role': 'user'}]
len of messages: 3


In [6]:
pprint.pprint(assistant.chat_messages[user_proxy])

[{'content': 'Message num: 0', 'role': 'user'},
 {'content': 'Message num: 1', 'role': 'user'},
 {'content': 'Message num: 2', 'role': 'user'},
 {'content': 'Message num: 3', 'role': 'user'},
 {'content': 'Message num: 4', 'role': 'user'},
 {'content': 'Message num: 5', 'role': 'user'},
 {'content': 'Message num: 6', 'role': 'user'},
 {'content': 'Message num: 7', 'role': 'user'},
 {'content': 'Message num: 8', 'role': 'user'},
 {'content': 'Message num: 9', 'role': 'user'},
 {'content': 'Message num: 10', 'role': 'user'},
 {'content': 'I see that the message numbers are increasing by 1 each time. To '
             'continue the pattern, the next message number should be 11.\n'
             '\n'
             'TERMINATE',
  'role': 'assistant'}]
